# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [38]:
!pip install gurobipy

# load require packages

In [39]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [40]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,170))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
157
158


### Parameters

In [41]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13

# Store model parameters
intercept = 1.978242858
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
season_coeff = {
    1: 0,
    2: -0.562046910,
    3: 0.087545274,
    4: -0.402637480,
    5: -0.027326010,
    6: 0.004349885,
    7: -0.036102297,
    8: -0.069280527,
    9: 0.160276197,
    10: 1.104208897,
    11: 1.122711287,
    12: 1.176802194,
    13: 0.947945548
}

In [42]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: np.float64(1.0), 158: np.float64(1.0), 159: np.float64(1.0), 160: np.float64(1.0), 161: np.float64(2.0), 162: np.float64(2.0), 163: np.float64(2.0), 164: np.float64(2.0), 165: np.float64(3.0), 166: np.float64(3.0), 167: np.float64(3.0), 168: np.float64(3.0), 169: np.float64(4.0)}


## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 169  and demand in weeks 157 – 169
######  Prices; 𝑝_𝑡  for  t = 157, … , 169   and   𝑑_𝑡  for  t = 157, … , 169
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝169𝑑169
#### Objective function,  sales revenue in next quarter (weeks 157 – 169)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:
#### 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13


In [43]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

In [44]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [45]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>,
 164: <gurobi.Var *Awaiting Model Update*>,
 165: <gurobi.Var *Awaiting Model Update*>,
 166: <gurobi.Var *Awaiting Model Update*>,
 167: <gurobi.Var *Awaiting Model Update*>,
 168: <gurobi.Var *Awaiting Model Update*>,
 169: <gurobi.Var *Awaiting Model Update*>}

In [46]:
# Set objective function
# Example first 2 lines fully written out,
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [47]:
mod1.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 13 columns and 0 nonzeros
Model fingerprint: 0xce1708e2
Model has 36 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 13 columns, 0 nonzeros
Presolved model has 36 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 12
 AA' NZ     : 6.600e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.33782030e+05  1.74140470e+05  2.0

In [48]:
p[157] # to see 1st prediction

<gurobi.Var C0 (value 0.9341427487341524)>

In [49]:
p[158] # to see 2nd prediction

<gurobi.Var C1 (value 0.9129826524411168)>

In [50]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x
df1

,price
157,0.934143
158,0.912983
159,0.879799
160,0.848879
161,0.735315
162,0.713867
163,0.713289
164,0.730312
165,0.845207
166,0.847602


In [51]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [52]:
p_ladder = [1.0, 0.95, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [53]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [54]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [55]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [56]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [57]:
mod2.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 91 columns and 169 nonzeros
Model fingerprint: 0xbaf8ae07
Model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 8.4722613
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 26 rows, 78 columns, 143 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 65 integer (65 binary)

Root relaxation: objective 1.046838e+01, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

In [58]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,0.95
158,0.95
159,0.85
160,0.85
161,0.75
162,0.75
163,0.75
164,0.75
165,0.85
166,0.85


In [59]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 3 - at most 4 promotions

In [60]:
# Create model object
mod3 = gp.Model ("price_model_3")

In [61]:
# Define decision variables
p = mod3.addVars(weeks)
x = mod3.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [62]:
# Set objective function
obj_fn = mod3.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [63]:
# Select price ladder value
constr_select_ladder = mod3.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod3.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod3.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)

In [64]:
mod3.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 27 rows, 91 columns and 234 nonzeros
Model fingerprint: 0x7106d0bb
Model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 8.5007336
Presolve time: 0.00s
Presolved: 27 rows, 91 columns, 221 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)

Root relaxation: objective 1.040803e+01, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [65]:
df3 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df3.loc[w,"price"] = p[w].x
df3

,price
157,1.0
158,1.0
159,1.0
160,1.0
161,1.0
162,1.0
163,0.85
164,0.85
165,1.0
166,1.0


In [66]:
fig = px.line(df3, x=df3.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 4 - full model

In [67]:
# Create model object
mod4 = gp.Model ("price_model_4")

In [68]:
# Define decision variables
p = mod4.addVars(weeks)
x = mod4.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [69]:
# Set objective function
obj_fn = mod4.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [70]:
# Select price ladder value
constr_select_ladder = mod4.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod4.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod4.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)
# No consecutive promotions
constr_no_consec_promo = mod4.addConstrs((sum(x[w,k] for k in p_ladder[1:]) + sum(x[w+1,k] for k in p_ladder[1:]) <= 1) for w in weeks[:-1])

In [71]:
mod4.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 39 rows, 91 columns and 354 nonzeros
Model fingerprint: 0x4e11b14e
Model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 8.4405578
Presolve time: 0.00s
Presolved: 39 rows, 91 columns, 341 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 78 integer (78 binary)

Root relaxation: objective 1.036734e+01, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [72]:
df4 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df4.loc[w,"price"] = p[w].x
df4

,price
157,1.0
158,1.0
159,1.0
160,1.0
161,1.0
162,0.85
163,1.0
164,0.85
165,1.0
166,1.0


In [73]:
fig = px.line(df4, x=df4.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

### End of script
---
